# Stock Trading Bot Ayyyyy

In [16]:
#This is just importing certain libraries we will use consistently throughouit the porject
import pandas as pd
import numpy as np
import random
import requests

#### In order to access the data from the API, we need the following:
- api key: It's generated by the platform as a hash code that is unique to each user so it knows that they are authenticated to use the service
- Base URL: For all calls to the api, we use the same base url(`https://www.alphavantage.co/query?`). However, in order to get more specified data, such as weekly data or data from a certain stock ticker, we need to add that to the url as parameters. Since all will use the `base_url`, we instantiate a global variable that be used wherever we need in any function and so we don't have to keep re-pasting it.
- tickers: this is just an array of the s&p 500 tickers. We will obviously need an array of them because we want to examine multiple stocks, not just one

In [71]:
api_key = "FI0W1W7TPDCRLO18"
base_url = "https://www.alphavantage.co/query?"
tickers = ['IBM', 'AAPL'] # this is just a test for now. Will add many more later, possibly thru another api

In [72]:
def intraday_req(ticker_arr):
    """This function takes in 1 parameter, the ticker array, and makes a request to the api with every single stock ticker symbol
    that is inside of the tickers array we creatred in the above cell."""
    function = "TIME_SERIES_INTRADAY"
    for ticker in tickers:
        url = f"{base_url}function={function}&symbol={ticker}&interval=60min&apikey={api_key}"
        response = requests.get(url)
        data = response.json()
        print(data)

In [73]:
intraday_req(tickers)

{'Meta Data': {'1. Information': 'Intraday (60min) open, high, low, close prices and volume', '2. Symbol': 'IBM', '3. Last Refreshed': '2023-06-16 19:00:00', '4. Interval': '60min', '5. Output Size': 'Compact', '6. Time Zone': 'US/Eastern'}, 'Time Series (60min)': {'2023-06-16 19:00:00': {'1. open': '137.5200', '2. high': '137.5200', '3. low': '137.5000', '4. close': '137.5000', '5. volume': '374'}, '2023-06-16 17:00:00': {'1. open': '137.6400', '2. high': '137.6400', '3. low': '137.5800', '4. close': '137.5800', '5. volume': '4590'}, '2023-06-16 16:00:00': {'1. open': '137.5600', '2. high': '138.0000', '3. low': '137.4800', '4. close': '137.4800', '5. volume': '289011'}, '2023-06-16 15:00:00': {'1. open': '137.7400', '2. high': '137.9900', '3. low': '137.4700', '4. close': '137.5500', '5. volume': '966229'}, '2023-06-16 14:00:00': {'1. open': '137.8300', '2. high': '137.9300', '3. low': '137.6900', '4. close': '137.7400', '5. volume': '325190'}, '2023-06-16 13:00:00': {'1. open': '138

In [78]:
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IM&interval=5min&apikey={api_key}'
r = requests.get(url)
data = r.json()

print(data)

{'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.'}


##### Here is an example of an error message. This is due to the fact that I changed the ticker to be one from a fake company `IM`, which isn't a real ticker on s&p 500. Sometimes errors and bugs can be hella easy to spot or hard as sheesh and take hours, but just to expose y'all to sum error messaging and handling